<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setting" data-toc-modified-id="Setting-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setting</a></span></li></ul></div>

## Setting

In [4]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import demoji
from konlpy.tag import Okt
from collections import Counter
import kss
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from hanspell import spell_checker # 네이버 맞춤법 검사기 파이썬용
import itertools
from kiwipiepy import Kiwi
import openai
import time
import datetime
import csv
import pymssql
import sqlalchemy as sa
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy import insert
from sqlalchemy import create_engine
from sqlalchemy import MetaData, Table, Column, Integer, String

kiwi = Kiwi()
okt = Okt()

import warnings
warnings.filterwarnings(action='ignore')

In [5]:
# DB 접근 정보
connection_uri = sa.engine.url.URL.create(
    "mssql+pyodbc",
    username="BI_APPUSER",
    password="dkagh1234.",
    host="KRVAEEISDBSQL",
    database="AUD", 
    query={"driver": "ODBC Driver 17 for SQL Server"},
    )
# ESG102PC295
engine = sa.create_engine(connection_uri, fast_executemany=True)
conn = engine.connect()

In [6]:
raw_review = pd.read_sql_table('bi_hotel_raw_review', conn)
review_preprocessed = pd.read_sql_table('bi_hotel_review_preprocessed', conn)
gpt_answer = pd.read_sql_table('bi_hotel_review_gpt_answer', conn)

In [7]:
df = pd.merge(raw_review, review_preprocessed,on='review_index' )
df = pd.merge(df,gpt_answer,on= 'checked_spelling')

In [45]:
# 리스트 값 카운트 긍정 단어, 부정 단어 
def word_counter(word_df):
    # null type 제외하고 다시 리스트화시킴
    word_df = word_df[~word_df['pos_word_stem'].isnull()]['pos_word_stem'].apply(lambda x:x.split(','))
    word_list = list(zip(word_df))
    word_list = list(itertools.chain(*word_list)) # 이중으로 쌓여있는 구조라 두번 flatten 작업이 필요함
    word_list = list(itertools.chain(*word_list))
    
    counter = Counter(word_list)
    return counter.most_common()

In [47]:
check_df = df.loc[(df['brunch']=='여의도') & (df['객실']==1)& (df['w_date']>='2022-01-01') & (df['w_date'] <='2023-03-01')]
word_counter(check_df)

[('', 8),
 ('조식', 8),
 ('해피아워', 6),
 ('컨디션', 5),
 ('친절하다', 5),
 ('최고', 4),
 ('괜찮다', 4),
 ('깨끗하다', 4),
 ('라운지', 4),
 ('편하다', 4),
 ('넓다', 3),
 ('만족스럽다', 3),
 ('좋아하다', 3),
 ('가능', 3),
 ('상태', 3),
 ('위치', 3),
 ('한강', 3),
 ('따뜻하다', 2),
 ('먹다', 2),
 ('브룸', 2),
 ('조절', 2),
 ('야경', 2),
 ('깔끔하다', 2),
 ('객실', 2),
 ('업그레이드', 2),
 ('아이', 1),
 ('자기', 1),
 ('큐티브', 1),
 ('성비', 1),
 ('커피차', 1),
 ('쿠키', 1),
 ('무제한', 1),
 ('와인', 1),
 ('맥주', 1),
 ('칵테일', 1),
 ('뷔페', 1),
 ('젝티', 1),
 ('결혼기념일', 1),
 ('여행', 1),
 ('잘나오다', 1),
 ('칭찬', 1),
 ('쉬다', 1),
 ('넷플릭스', 1),
 ('유튜브', 1),
 ('티타임', 1),
 ('코스', 1),
 ('이용', 1),
 ('가성', 1),
 ('시원하다', 1),
 ('관리', 1),
 ('인테리어', 1),
 ('맛집', 1),
 ('코엑스', 1),
 ('탁월하다', 1),
 ('선택', 1),
 ('캐릭터', 1),
 ('벽지', 1),
 ('바닥', 1),
 ('텐트', 1),
 ('딸아이', 1),
 ('춥다', 1),
 ('냉방', 1),
 ('세기', 1),
 ('키즈', 1),
 ('귀엽다', 1),
 ('족편', 1),
 ('안좋다', 1),
 ('큐티', 1),
 ('예쁘다', 1),
 ('만족하다', 1),
 ('사용', 1),
 ('가격', 1),
 ('대비', 1),
 ('욕조', 1),
 ('비데', 1),
 ('간혹', 1),
 ('맛있다', 1),
 ('직원', 1),
 ('세심', 1),
 ('챙

In [ ]:

## Tableau 용 
def get_output_schema():       
    return pd.DataFrame({
        'Word' : prep_string(),
        'Count' : prep_int()
    })

def word_counter(word_col):
    word_list = list(zip(word_col))
    word_list = list(itertools.chain(*word_list)) 
    
    counter = Counter(word_list)
    result =  pd.DataFrame(counter.most_common(), columns=['Word', 'Count'])
    return pd.DataFrame({
        'Word' : result['Word']
        'Count':result['Count']
    })